In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib

# check xgboost version
from xgboost import XGBRegressor

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
suf = "_normal_xgb_01_11"

returns = pd.read_csv(f"{models_dir}/test_returns_predictions{suf}.csv").rename(
    columns={
        "end_station_cluster": "start_station_cluster",
        "pred": "returns"
        }
    )[['started_at_hourly', 'returns', 'start_station_cluster']]

In [ ]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')
df = df.merge(returns, on=['started_at_hourly', 'start_station_cluster'], how='left').fillna(0)

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [ ]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

In [ ]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# GNN Variance Embedding

In [ ]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            "temp",
            "dwpt",
            "rhum",
            "prcp",
            "wdir",
            "wspd",
            "pres",
            "coco",
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            "mean_gnn_cluster_demand_1h",
            "total_gnn_cluster_demand_1h",
            "total_demand_1h",
            "demand_degrees_1h",
            "returns"
]
FEATURES.extend([f"dim_var_{i}" for i in range(50)])

## XGBoost

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.519233287292934
RMSE: 1.8759619631786073
MAE: 0.994659759299992
MAPE: 0.39427954321027453

Non-zero
MSE: 7.178668166392236
RMSE: 2.67930367192527
MAE: 1.6885697091754872
MAPE: 0.35389761853210966

Zeros
MSE: 0.5271638124463442
RMSE: 0.7260604743727235
MAE: 0.42729707956682533
MAPE: 0.42729707956682533


In [ ]:
#11-09 scaled
3.526444658025326
1.877883025650247
0.39191876965741634

#11-09 unscaled
3.6554374527124036
1.9119198342797754
0.4147471104914078

#11-28
3.5321116400032615
1.8793912950748872
0.3998485691043102

0.4147471104914078

In [ ]:
suf = "_l_xgb_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.194217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16454
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 87
[LightGBM] [Info] Start training from score 1.005143
overall
MSE: 3.615246555019159
RMSE: 1.901380171091294
MAE: 1.0081537181464293
MAPE: 0.3988012639404724

Non-zero
MSE: 7.383863942452731
RMSE: 2.7173266168152717
MAE: 1.710606327797798
MAPE: 0.3559885502656281

Zeros
MSE: 0.5339060147848538
RMSE: 0.7306887263293815
MAE: 0.43380629011363325
MAPE: 0.43380629011363325


In [ ]:
suf = "_l_lgbm_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='rf')

overall
MSE: 3.8007531649426345
RMSE: 1.9495520421221473
MAE: 1.0470548391668155
MAPE: 0.4363587570694511

Non-zero
MSE: 7.646508376188069
RMSE: 2.765232065521458
MAE: 1.717827751045442
MAPE: 0.3602230285760248

Zeros
MSE: 0.6563423049434458
RMSE: 0.8101495571457444
MAE: 0.49860973206875425
MAPE: 0.49860973206875425


In [ ]:
#11_01
3.6042873699256828
1.8984960810930538
0.4084309933647605

#11_28
3.776596398611332
1.9433467005687204
0.43658466526229694

0.4084309933647605

In [ ]:
suf = "_l_rf_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model